# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [4]:
import gym
import random
import numpy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from collections import deque  # for memory
from tqdm import tqdm          # for progress bar

How the game looks (without our agent):

In [5]:
def rnd_game():
    env = gym.make('CartPole-v1', render_mode='human')
    for _ in tqdm(range(10)):
        state, _ = env.reset()
        done = False
        while not done:
            action = env.action_space.sample()
            next_state, reward, done, _, _ = env.step(action)
    env.close()

# rnd_game()

## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

$ \begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align} $

However, we don't know $ Q^* $. So, we use neural network as a approximators, we can simply create one and train it to resemble $ Q^* $.

For our training update rule, we'll use a fact that every $ Q $
function for some policy obeys the Bellman equation:

$ \begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align} $

The difference between the two sides of the equality is known as the temporal difference error, $ \delta $:

$ \begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align} $

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.

In [28]:
class Model(nn.Module):
    def __init__(self, observation_size, action_size):
        super(Model, self).__init__()
        # initialise layers here
        self.layer1 = nn.Linear(observation_size, 10)    # create dense layer 1
        self.layer2 = nn.Linear(10, action_size)         # create dense layer 2
 
    # uso il forward solo con x che è una lista di interi o float
    def forward(self, x):
        # send x through the network

        x = torch.tensor(x, dtype=torch.float32)
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        
        return x

    def predictBestReward(self, x):
        x = self.forward(x)            # send x through neural net
        # res,_ = torch.max(x, dim=0)
        return torch.max(x)     # predict the best reward

    def predictBestAction(self, x):
        x = self.forward(x)               # send x through neural net
        return torch.argmax(x, dim=0)     # predict the best action


### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [26]:
class Agent:
    def __init__(self, observation_size, action_size):

        self.observation_size=observation_size
        self.action_size = action_size

        self.criterion = nn.MSELoss()
        self.model = Model(observation_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)

        # memory that stores N most new transitions
        self.memory_size = 1000
        self.memory = deque()
        self.memory_full = False
        
        # good place to store hyperparameters as attributes
        self.gamma = 0.9

    def remember(self, state, action, reward, next_state, done):
        sars = {"state": state,
                "action": action,
                "reward": reward,
                "next_state": next_state,
                "done": done}
        # add to memory
        self.memory.append(sars)
        # remove oldest value if the memory is full
        if self.memory_full:
            self.memory.popleft()
        elif len(self.memory) == self.memory_size :
            self.memory_full = True

    def act(self, state):
        # return an action from the model
        # out = self.model.forward(state)
        # return torch.argmax(out, dim=0)
        return self.model.predictBestAction(state)

    def replay(self, batch_size):
        # update model based on replay memory
        # you might want to make a self.train() helper method
        batch = random.sample(self.memory, batch_size)
        for sars in batch:
            self.train(sars)

    def train(self, sars):
        
        self.optimizer.zero_grad()                                # clean gradients of parameters

        pred = self.model.forward(sars["state"])[sars["action"]]        # take the Q value of the action

        if sars["done"] == True:
            y = sars["reward"]
            y = torch.tensor(y, dtype = torch.float32)
        else:
            max_reward = self.model.predictBestReward(sars["next_state"])
            #out_tensor = self.model.forward(sars["next_state"])
            # The maximum of the rewards achievable from the next_state, 
            # with argmax I take the best action, with max the reward of the best action
            #max_value = torch.max(out_tensor).item() 
            y = sars["reward"] + self.gamma * max_reward
        loss = self.criterion(pred, y)          # calculate loss with respect to prediction
        loss.backward()                         # calculate gradients of model.parameters() with respect to loss
        self.optimizer.step()                   # update parameters with respect to gradients



### Main Training loop
---
Make a function that takes an environment and an agent, and runs through $ n $ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [8]:
def train(env, agent: Agent, episodes=1000, batch_size=64):  # train for many games
    for _ in tqdm(range(episodes)):
        state, _ = env.reset()
        done = False
        while not done:
            # 1. make a move in game.
            tradeoff = random.uniform(0,1)
            if tradeoff > 0.3:
                action = agent.model.predictBestAction(state).item()
            else:
                action = env.action_space.sample()
            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, done, _, _ = env.step(action)

            # 2. have the agent remember stuff.
            agent.remember(state, action, reward, new_state, done)

            # 3. update state
            state = new_state

            # 4. if we have enough experiences in our memory, learn from a batch with replay.
            if len(agent.memory) >= batch_size:
                agent.replay(batch_size)
    env.close()

### Putting it together
---
We train an agent on the environment:


In [30]:
env = gym.make('CartPole-v1', render_mode='human')
agent = Agent(env.observation_space.shape[0], env.action_space.n)
train(env, agent)
torch.save(agent.model.state_dict(), 'modelCartPole.pth')

  1%|▏         | 13/1000 [00:35<44:21,  2.70s/it] 


KeyboardInterrupt: 

## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/